# Random Forest Regression

## Importing the libraries

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import math
import gc

## Importing the dataset

In [12]:
train = pd.read_csv('worked/train_worked.csv', index_col='ID')
y_train = train.iloc[:, 1].values
train.pop('target')
catFeatures = ['location']
train = pd.get_dummies(train, columns=catFeatures)
train['target'] = y_train
x_train = train.iloc[:, :-1].values

test = pd.read_csv('worked/test_worked.csv', index_col='ID')
test.pop('target')
catFeatures = ['location']
test = pd.get_dummies(test, columns=catFeatures)
x_test = test.iloc[:, :].values


In [13]:
train.head(5)

,max_temp,min_temp,mean_temp,std_temp,var_temp,median_temp,ptp_temp,max_precip,min_precip,mean_precip,...,std_atmos_press,var_atmos_press,median_atmos_press,ptp_atmos_press,location_A,location_B,location_C,location_D,location_E,target
ID,,,,,,,,,,,,,,,,,,,,,
ID_train_0,27.208333,19.275000,22.299527,2.594011,6.728895,21.358333,7.933333,0.000,0.0,0.000000,...,0.070374,0.004952,87.762083,0.257500,0,0,1,0,0,45.126304
ID_train_1,33.616667,17.983333,24.679063,4.266955,18.206903,23.791667,15.633333,0.561,0.0,0.007025,...,0.156000,0.024336,90.429167,0.668333,0,0,0,1,0,79.131702
ID_train_10,31.841667,18.458333,24.112317,3.776377,14.261020,23.304167,13.383333,7.804,0.0,0.151767,...,0.179481,0.032213,88.425000,0.830833,1,0,0,0,0,32.661304
ID_train_100,27.491667,16.941667,20.845273,3.080140,9.487259,19.541667,10.550000,25.787,0.0,1.127273,...,0.161758,0.026166,88.400000,0.720000,1,0,0,0,0,53.850238
ID_train_1000,28.750000,17.525000,21.870732,3.286870,10.803515,20.950000,11.225000,0.136,0.0,0.005100,...,0.118369,0.014011,88.552500,0.450833,1,0,0,0,0,177.418750


In [14]:
train.columns

Index(['max_temp', 'min_temp', 'mean_temp', 'std_temp', 'var_temp',
       'median_temp', 'ptp_temp', 'max_precip', 'min_precip', 'mean_precip',
       'std_precip', 'var_precip', 'median_precip', 'ptp_precip',
       'max_rel_humidity', 'min_rel_humidity', 'mean_rel_humidity',
       'std_rel_humidity', 'var_rel_humidity', 'median_rel_humidity',
       'ptp_rel_humidity', 'max_wind_dir', 'min_wind_dir', 'mean_wind_dir',
       'std_wind_dir', 'var_wind_dir', 'median_wind_dir', 'ptp_wind_dir',
       'max_wind_spd', 'min_wind_spd', 'mean_wind_spd', 'std_wind_spd',
       'var_wind_spd', 'median_wind_spd', 'ptp_wind_spd', 'max_atmos_press',
       'min_atmos_press', 'mean_atmos_press', 'std_atmos_press',
       'var_atmos_press', 'median_atmos_press', 'ptp_atmos_press',
       'location_A', 'location_B', 'location_C', 'location_D', 'location_E',
       'target'],
      dtype='object')

In [15]:
train.shape

(15539, 48)

In [16]:
x_train

array([[27.21, 19.27, 22.3 , ...,  1.  ,  0.  ,  0.  ],
       [33.62, 17.98, 24.68, ...,  0.  ,  1.  ,  0.  ],
       [31.84, 18.46, 24.11, ...,  0.  ,  0.  ,  0.  ],
       ...,
       [34.11, 16.95, 24.64, ...,  0.  ,  1.  ,  0.  ],
       [30.9 , 15.89, 23.47, ...,  0.  ,  1.  ,  0.  ],
       [30.45, 19.03, 24.01, ...,  1.  ,  0.  ,  0.  ]])

## Splitting the dataset into the Training set and Test set

In [35]:
from sklearn.model_selection import train_test_split
x_trainer, x_tester, y_trainer, y_tester = train_test_split(x_train, y_train, test_size=0.20)

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor(n_estimators = 150, random_state=32)
reg.fit(x_trainer, y_trainer)
cvs = cross_val_score(reg, x_trainer, y_trainer, cv=10)

print(cvs)
print(np.mean(cvs))

[0.53 0.53 0.58 0.6  0.57 0.56 0.61 0.6  0.57 0.59]
0.5734073987064299


In [36]:
y_pred = reg.predict(x_tester)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_tester.reshape(len(y_tester),1)),1))

[[105.14  85.61]
 [ 15.82   5.8 ]
 [ 52.27  46.15]
 ...
 [ 40.82  45.57]
 [ 79.14  41.57]
 [ 36.51  37.33]]


In [37]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_pred, y_tester))
print('Mean Squared Error:', metrics.mean_squared_error(y_pred, y_tester))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_pred, y_tester)))

Mean Absolute Error: 15.751726317698948
Mean Squared Error: 661.8787556598247
Root Mean Squared Error: 25.72700440509592


In [10]:
test.head()

,max_temp,min_temp,mean_temp,std_temp,var_temp,median_temp,ptp_temp,max_precip,min_precip,mean_precip,...,mean_atmos_press,std_atmos_press,var_atmos_press,median_atmos_press,ptp_atmos_press,location_A,location_B,location_C,location_D,location_E
ID,,,,,,,,,,,,,,,,,,,,,
ID_test_0,26.858333,17.050000,21.389532,2.731833,7.462909,20.908333,9.808333,5.728,0.0,0.204165,...,87.748506,0.132764,0.017626,87.752500,0.546667,0,0,1,0,0
ID_test_1,26.991667,17.275000,21.302824,2.289268,5.240748,20.716667,9.716667,15.831,0.0,0.208719,...,87.936963,0.113756,0.012940,87.928333,0.609167,0,0,1,0,0
ID_test_10,29.850000,17.108333,22.144421,2.943047,8.661528,21.458333,12.741667,1.327,0.0,0.040190,...,90.521357,0.136316,0.018582,90.536667,0.670000,0,0,0,1,0
ID_test_100,27.325000,18.633333,22.361639,2.480003,6.150415,21.616667,8.691667,3.604,0.0,0.035686,...,87.754848,0.125583,0.015771,87.726667,0.518333,0,0,1,0,0
ID_test_1000,31.091667,16.908333,23.560243,4.136978,17.114585,22.825000,14.183333,0.000,0.0,0.000000,...,90.475030,0.161193,0.025983,90.487273,0.692500,0,0,0,1,0


## Training the Random Forest Regression model on the whole dataset

## Predicting the Test set results

## Evaluating the Model Performance